In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
header={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}

## Function to scrape product name and asin no (as the file path is similar)

In [3]:
def Search(search_query):
    url="https://www.amazon.in/s?k="+search_query #search link
    page=requests.get(url,headers=header)  
    if page.status_code==200:
        return page                                #returns the page if there is no error
    else:
        return "Error"



## Function to scrape link of the All customer reviews to acess all the reviews

In [4]:
def url_link(query):
    url="https://www.amazon.in/dp/"+query     #search link
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page                            #return the page if there is no error
    else:
        return "Error"    

## Function to scrape product reviews 

In [5]:
def content(query):
    url="https://www.amazon.in/"+query
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error" 

## This saves all the product name and the asin no in a list

In [6]:
product_Name=[]
Asin_no=[]
for i in range(1,16):
    response=Search("smartphones&page="+str(i))
    soup=BeautifulSoup(response.content)
    for i in soup.findAll('span',attrs={'class':'a-size-medium a-color-base a-text-normal'}): 
        product_Name.append(i.text)
    for i in soup.find_all('div',attrs={'data-asin':True}):
        Asin_no.append(i['data-asin'])

In [7]:
product_Name

['Apple iPhone 7 (32GB) - Black',
 'Apple iPhone 11 (128GB) - Black (Includes EarPods, Power Adapter)',
 'Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)',
 'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Samsung Galaxy M01 (Black, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Samsung Galaxy M21 (Raven Black, 4GB RAM, 64GB Storage)',
 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)',
 'TECNO Spark 6 Air (Comet Black, 3GB RAM, 64GB Storage)',
 'OnePlus 7T (Glacier Blue, 8GB RAM, Fluid AMOLED Display, 256GB Storage, 3800mAH Battery)',
 'Redmi Note 9 (Pebble Grey, 4GB RAM 64GB Storage) - 48MP Quad Camera & Full HD+ Displ

In [8]:
Asin_no

['',
 'B07HGJJ559',
 'B07HGH8D2R',
 'B07HGH88GL',
 'B01LZKSVRB',
 'B07XVLW7YK',
 'B07HGJJ559',
 'B07X4R63DF',
 'B089MQ622N',
 '',
 'B077PWBC78',
 'B07SBK1WQQ',
 'B07HGJKDQL',
 'B07DJ8K2KT',
 'B07SDPJ4XJ',
 'B086KCCMCP',
 'B07HGH8D2R',
 'B07WPVLKPW',
 'B07HGGYWL6',
 'B08HX4RKR1',
 'B07DJLVJ5M',
 'B086977TR6',
 'B07X3P1N3H',
 'B077PWBC78',
 'B07X1KT6LD',
 'B081FQ2X3T',
 'B07X1KSLC3',
 'B07X1KSWZ3',
 'B07HGH8ML7',
 'B077PWJRFH',
 'B086KCCMBT',
 '',
 '',
 '',
 '',
 '',
 '',
 'B085J1J32G',
 'B085J1X78K',
 'B085J1868F',
 'B01LZKSVRB',
 'B07XVLW7YK',
 'B085J1868F',
 'B08695ZSP6',
 'B07HGH8ML7',
 'B07DJCJBBN',
 'B086KFV4L4',
 'B08696ZMPF',
 'B07HGJKDQL',
 'B085J1JDB7',
 'B07HGH8ML7',
 'B07DJLVJ5M',
 'B078BN2H3R',
 'B077PWK5BY',
 'B07HGH88GL',
 'B086KDZ13S',
 'B0756ZFXWP',
 'B086977J3K',
 'B07SBK1WQQ',
 'B07KXC7WQZ',
 'B08HX4RKR1',
 'B08696XB45',
 '',
 '',
 '',
 '',
 '',
 '',
 'B07DJLVJ5M',
 'B078BNQ318',
 'B07DJ8K2KT',
 'B01LZKSVRB',
 'B07XVLW7YK',
 'B07SBK1WQQ',
 'B077PWBC78',
 'B07PP2K69Z',


## This takes all the links and ignores if the asin no is blank

In [9]:
Link=[]
for i in range(len(Asin_no)):
    if Asin_no[i]!='':
        link_response=url_link(Asin_no[i])       #iterates through Asin no to access the product
        soup=BeautifulSoup(link_response.content)
        for i in soup.findAll('a',attrs={'data-hook':'see-all-reviews-link-foot'}):
            Link.append(i['href'])               #saves the footer link of all reviews to a list

In [10]:
len(Link)

247

## For each product we are scraping 15 pages of reviews and ratings

In [11]:
reviews=[]
ratings=[]
for k in range(len(Link)):                                      #helps to iterate for different products
    for i in range(1,15):                               #each product , it scrapes 15 pages of reviews
        cont_response=content(Link[k]+'&pageNumber='+str(i))    #iterates through multiple pages of the reviews
        soup=BeautifulSoup(cont_response.content)
        for j in soup.findAll("span",attrs={'data-hook':'review-body'}):
            reviews.append(j.text)                        #saves review content
        for star in soup.findAll("i",attrs={'data-hook':"review-star-rating"}):
            ratings.append(star.text)                 #saves ratings of the respective reviews
            

## saving the file in dataframe

In [34]:
new_reviews = []
for review in reviews:
    new_reviews.append(review.strip())

In [35]:
df=pd.DataFrame(columns=['Smartphone_Review','Rating']) 


In [42]:
df.to_csv('Smartphone_Review.csv',index=False)   #saving it into a dataframe

In [37]:
df=df.drop_duplicates()                   #removing duplicates

In [40]:
df['Smartphone_Review']=new_reviews
df['Rating']=ratings

In [41]:
df

,Smartphone_Review,Rating
0,Update (15/08/2020):Never give a chance to reg...,4.0 out of 5 stars
1,As a title it is obviously a monsterAnd very v...,5.0 out of 5 stars
2,"Brilliant camera, huge battery life and brilli...",5.0 out of 5 stars
3,I am writing my review after using it for 6 da...,3.0 out of 5 stars
4,"Defective product received, it gets 8 to 12 ho...",1.0 out of 5 stars
...,...,...
29531,After using samsung m21 and verifying its fea...,3.0 out of 5 stars
29532,It is the worst mobile I ever had. Company cla...,1.0 out of 5 stars
29533,The battery is not actually 6000mAh but slight...,4.0 out of 5 stars
29534,I had to wait for long for this mobile as I wa...,5.0 out of 5 stars
